In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!pip install mtcnn

In [0]:
import pickle
import keras
from keras.models import load_model
import cv2
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import os
from mtcnn import MTCNN

In [0]:
folder_path = '/content/drive/My Drive/week10/face_detection'

In [0]:
# read db
with open(os.path.join(folder_path, 'db.pickle'), 'rb') as file:
    db = pickle.load(file)
embeddings = db['embeddings']
names = db['names']

In [0]:
raw_img = cv2.imread(os.path.join(folder_path, 'face1.jpg'))[:,:,::-1]
plt.imshow(raw_img)

In [0]:
detector = MTCNN()
feature_extractor = load_model(os.path.join(folder_path, 'facenet.h5'))

In [0]:
# detect face
def detect_faces(img):
    face_imgs = []
    results = detector.detect_faces(img)
    # extract the bounding box from the first face
    print('face count: ', len(results), results)
    for i in range(len(results)):
        x1, y1, width, height = results[i]['box']
        x2, y2 = x1 + width, y1 + height
        patch = img[y1:y2, x1:x2] # crop face
        face_imgs.append(patch)
    return face_imgs

faces = detect_faces(raw_img)

In [0]:
# all face shapes
for f in faces:
  print(f.shape)

In [0]:
# show all faces
for face in faces:
    plt.imshow(face)
    plt.show()

In [0]:
# Standardization
def preprocess(imgs): 
    for i in range(imgs.shape[0]):
        # standardization
        img = imgs[i]
        mean, std = img.mean(), img.std()
        img = (img - mean) / std
        imgs[i] = img
    return imgs

In [0]:
face = faces[0]
face = cv2.resize(face, (160, 160))
model_input = np.zeros((1, 160, 160, 3))
model_input[0] = face
model_input = preprocess(model_input)
query_embedding = feature_extractor.predict(model_input)

In [0]:
query_embedding = query_embedding[0]
query_embedding

In [0]:
def euclidean_distance(x, y):
    sum_square = np.sum(np.square(x - y), keepdims=True)
    return np.sqrt(sum_square)

In [0]:
# Get all distances from database embeddings 
distances = np.zeros((len(embeddings)))
for i, embed in enumerate(embeddings):
    distance = euclidean_distance(embed, query_embedding)
    distances[i] = distance

In [0]:
print(distances[:10])
plt.hist(distances)

In [0]:
np.argmin(distances)

In [0]:
idx_min = np.argmin(distances)
distances[idx_min], names[idx_min]

In [0]:
def face_id(filename, IMG_SIZE=160):
    raw_img = cv2.imread(os.path.join(folder_path, filename))[:,:,::-1]
    faces = detect_faces(raw_img)
    if len(faces) == 0:
        print('No face')
        return
    else:
        # get face embeddings
        face = faces[0]


        face = cv2.resize(face, (IMG_SIZE, IMG_SIZE))
        model_input = np.zeros((1, IMG_SIZE, IMG_SIZE, 3))
        model_input[0] = face
        model_input = preprocess(model_input)
        query_embeddings = feature_extractor.predict(model_input)
        query_embedding = query_embeddings[0]
        
        # compute distance
        distances = np.zeros((len(embeddings)))
        for i, embed in enumerate(embeddings):
            distance = euclidean_distance(embed, query_embedding)
            distances[i] = distance
        plt.hist(distances)
        plt.show()
        # find min distance    
        idx_min = np.argmin(distances)
        distance, name = distances[idx_min], names[idx_min]
        print(name, distance)
        plt.imshow(raw_img)
        plt.show()

In [0]:
face_id('face5.jpg')